<a href="https://colab.research.google.com/github/gr1nick/lab_bigData/blob/master/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перед началом выполнения работы необходимо загрузить и предварительно оценить данные из файла Orders.xlsx для определения численных и категориальных переменных.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Функция масштабирования данных
def minmax_scale(data):
    min_vals = data.min(axis=0)
    max_vals = data.max(axis=0)
    scale = max_vals - min_vals
    return (data - min_vals) / scale, min_vals, scale

# Функция нормализации данных
def standardize(data):
    mean_vals = data.mean(axis=0)
    std_dev = data.std(axis=0)
    return (data - mean_vals) / std_dev, mean_vals, std_dev

def initialize_parameters(dim):
    """Инициализация весов и смещения нулями."""
    w = np.zeros((dim, 1))
    b = 0
    return w, b

def sigmoid(z):
    """сигмоидная функция."""
    z = np.clip(z, -500, 500)  # Ограничение значений z для предотвращения переполнения
    return 1 / (1 + np.exp(-z))

def compute_loss(y, y_pred):
    """Вычисление логистической потери."""
    m = len(y)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = -(1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    return loss

def gradient_descent(X, y, w, b, learning_rate, iterations):
    """Градиентный спуск для обновления параметров модели."""
    m = len(y)
    y = y.reshape(m, 1)  # Переформатирование для соответствия размерам

    for i in range(iterations):
        # Прямое распространение
        z = np.dot(X, w) + b
        y_pred = sigmoid(z)

        # Вычисление градиентов
        dw = (1 / m) * np.dot(X.T, (y_pred - y))
        db = (1 / m) * np.sum(y_pred - y)

        # Обновление параметров
        w -= learning_rate * dw
        b -= learning_rate * db

        # Опционально: Вывод функции потерь каждые 100 итераций
        if (i % 100 == 0) or (i % (iterations-1) == 0):
            print(f"Функция потерь при итерации {i}: {compute_loss(y, y_pred)}")

    return w, b

def softmax(z):
    """Реализация функции softmax на чистом Python."""
    # Вычисляем экспоненту для каждого элемента вектора z
    exp_scores = [np.exp(i) for i in z]
    # Суммируем все экспоненты для нормализации
    sum_exp_scores = sum(exp_scores)
    # Делим каждую экспоненту на сумму всех экспонент для получения вероятностей
    softmax_scores = [j / sum_exp_scores for j in exp_scores]
    return softmax_scores

def one_hot_encode(data, categorical_columns):
    '''Реализация функции one-hot-encode на чистом Python.'''
    # Копируем данные для избежания изменения исходного DataFrame
    data = data.copy()
    # Проходим по всем категориальным столбцам
    for column in categorical_columns:
        # Получаем уникальные значения для столбца
        unique_values = set(data[column])
        # Создаем словарь для новых столбцов однократного кодирования
        for unique_value in unique_values:
            # Создаем новый столбец для каждого уникального значения
            column_name = f"{column}_{unique_value}"
            data[column_name] = [1 if value == unique_value else 0 for value in data[column]]
        # Удаляем исходный категориальный столбец
        del data[column]
    return data

def predict(X, w, b):
    """Функция для предсказания классов с использованием обученных параметров."""
    z = np.dot(X, w) + b
    y_pred = sigmoid(z)
    y_pred_class = [1 if i > 0.5 else 0 for i in y_pred]  # Классификация как 1 или 0
    return np.array(y_pred_class)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Ncfu/BigData/Orders.xlsx'
#Загрузка данных из файла Excel
data = pd.read_excel(file_path)

# Отображение первых строк данных для оценки структуры и определения типов переменных
#data.head()

На основании предварительного просмотра данных, мы можем классифицировать переменные следующим образом:

Численные переменные: Row ID, Postal Code, Sales, Quantity, Discount, Profit.
Категориальные переменные: Order ID, Ship Mode, Customer ID, Customer Name, Segment, Country, City, State, Region, Product ID, Category, Sub-Category, Product Name.

In [ ]:
# Проверка на наличие пропущенных значений в данных
missing_values = data.isnull().sum()

# Отображение количества пропущенных значений по каждому столбцу, если они есть
missing_values[missing_values > 0]

Series([], dtype: int64)

In [ ]:
# Описательный анализ числовых переменных
descriptive_stats = data.describe()

# Построение корреляционной матрицы
correlation_matrix = data.corr()

# Вывод описательных статистик и корреляционной матрицы
#print(descriptive_stats)
#print(correlation_matrix)

<ipython-input-6-519a31e3f041>:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data.corr()


In [ ]:
# Список числовых признаков для гистограмм
numeric_features = ['Sales', 'Quantity', 'Discount', 'Profit']

# Построение гистограмм для каждого числового признака
for feature in numeric_features:
    plt.figure(figsize=(10, 4))
    plt.hist(data[feature], bins=50, edgecolor='k')
    plt.title('Гистограмма распределения для {}'.format(feature))
    plt.xlabel(feature)
    plt.ylabel('Количество')
    plt.show()

In [ ]:
# Определение численных и категориальных переменных
numeric_columns = ['Sales', 'Quantity', 'Discount', 'Profit']
categorical_columns = ['Ship Mode', 'Segment', 'Region', 'Category', 'Sub-Category']

In [ ]:
# Применение one-hot-encoding
data_encoded = one_hot_encode(data, categorical_columns)

In [ ]:
# Разделение на обучающую и тестовую выборки
split_idx = int(len(data_encoded) * 0.7)
data_train = data_encoded[:split_idx]
data_test = data_encoded[split_idx:]


In [ ]:
# Применение масштабирования и нормализации к обучающим данным
X_train_scaled, min_vals, scale = minmax_scale(data_train[numeric_columns].values)
X_train_normalized, mean_vals, std_dev = standardize(X_train_scaled)

In [ ]:
# Применение того же масштаба и нормализации к тестовым данным
X_test_scaled = (data_test[numeric_columns].values - min_vals) / scale
X_test_normalized = (X_test_scaled - mean_vals) / std_dev


In [ ]:
data_train = data_encoded[:split_idx].copy()
data_test = data_encoded[split_idx:].copy()

In [ ]:
# Обновляем обучающие и тестовые данные
data_train.loc[:, numeric_columns] = X_train_normalized
data_test.loc[:, numeric_columns] = X_test_normalized

In [ ]:
# Выбор целевой переменной и исключение нечисловых столбцов
median_profit = data_train['Profit'].median()
data_train['Profit_class'] = (data_train['Profit'] > median_profit).astype(int)
data_test['Profit_class'] = (data_test['Profit'] > median_profit).astype(int)

In [ ]:
X_train = data_train.drop(['Profit_class'], axis=1).values
X_train = data_train[numeric_columns].values
y_train = data_train['Profit_class'].values
X_test = data_test.drop(['Profit_class'], axis=1).values
X_test = data_test[numeric_columns].values
y_test = data_test['Profit_class'].values


In [ ]:
# Инициализация параметров
w, b = initialize_parameters(X_train.shape[1])

#градиентный спуск
w, b = gradient_descent(X_train, y_train, w, b, learning_rate=3, iterations=2000)

Функция потерь при итерации 0: 0.6931471805599453
Функция потерь при итерации 100: 0.3678653805756209
Функция потерь при итерации 200: 0.33112905135695064
Функция потерь при итерации 300: 0.30863538299323523
Функция потерь при итерации 400: 0.2918840130532637
Функция потерь при итерации 500: 0.2785159222158464
Функция потерь при итерации 600: 0.2674131477739694
Функция потерь при итерации 700: 0.2579387728641961
Функция потерь при итерации 800: 0.24969249151215606
Функция потерь при итерации 900: 0.24240572964284898
Функция потерь при итерации 1000: 0.23588940769522818
Функция потерь при итерации 1100: 0.2300053053252076
Функция потерь при итерации 1200: 0.2246492624760866
Функция потерь при итерации 1300: 0.21974080322265066
Функция потерь при итерации 1400: 0.21521645152786775
Функция потерь при итерации 1500: 0.21102526524217805
Функция потерь при итерации 1600: 0.20712575235934974
Функция потерь при итерации 1700: 0.2034836772394676
Функция потерь при итерации 1800: 0.2000704590267

In [ ]:
# Предсказание на тестовом наборе
y_pred_test = predict(X_test, w, b)

# Вычисление точности
accuracy = np.mean(y_pred_test == y_test)
print(f"Точность модели на тестовом наборе: {accuracy}")


Точность модели на тестовом наборе: 0.9359786595531844


Данные без предобработки

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
file_path = '/content/drive/My Drive/Ncfu/BigData/Orders.xlsx'
#Загрузка данных из файла Excel
data = pd.read_excel(file_path)


In [ ]:
# Определение численных
numeric_columns = ['Sales', 'Quantity', 'Discount', 'Profit', 'Row ID', 'Postal Code']
# Создание бинарной целевой переменной
data['Is_Profitable'] = (data['Profit'] > 0).astype(int)

In [ ]:
# Отбор только числовых столбцов для использования в модели
X = data[numeric_columns].values  # Преобразование DataFrame в numpy array
y = data['Is_Profitable'].values

In [ ]:
# Разделение на обучающую и тестовую выборки
split_idx = int(len(X) * 0.7)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

In [ ]:
# Инициализация параметров
w, b = initialize_parameters(X_train.shape[1])

In [ ]:
# Обучение модели
w, b = gradient_descent(X_train, y_train, w, b, learning_rate=3, iterations=2000)

Функция потерь при итерации 0: 6.710405114166012
Функция потерь при итерации 100: 6.710405114166012
Функция потерь при итерации 200: 27.793963163252634
Функция потерь при итерации 300: 6.690654105735796
Функция потерь при итерации 400: 6.710405114166012
Функция потерь при итерации 500: 27.803838438848036
Функция потерь при итерации 600: 6.710405114166012
Функция потерь при итерации 700: 6.710405114166012
Функция потерь при итерации 800: 27.601395289142342
Функция потерь при итерации 900: 6.710405114166012
Функция потерь при итерации 1000: 6.710405114166012
Функция потерь при итерации 1100: 6.710405114166012
Функция потерь при итерации 1200: 6.710405114166012
Функция потерь при итерации 1300: 6.710405114166012
Функция потерь при итерации 1400: 27.468079068604442
Функция потерь при итерации 1500: 6.700529609950904
Функция потерь при итерации 1600: 6.710405114166012
Функция потерь при итерации 1700: 6.710405114166012
Функция потерь при итерации 1800: 27.798900801050337
Функция потерь при 

In [ ]:
y_pred_test = predict(X_test, w, b)
accuracy = np.mean(y_pred_test == y_test)
print(f"Точность модели на тестовом наборе: {accuracy:.4f}")

Точность модели на тестовом наборе: 0.8079


softmax

In [ ]:

from math import exp
import pandas as pd
import numpy as np
file_path = '/content/drive/My Drive/Ncfu/BigData/Orders.xlsx'
#Загрузка данных из файла Excel
data = pd.read_excel(file_path)



In [ ]:
# Преобразование "Segment" в числовые метки
segments = {'Consumer': 0, 'Corporate': 1, 'Home Office': 2}
segment_labels = np.array(data['Segment'].apply(lambda x: segments[x]))

# Выделяем числовые признаки для использования в модели
features = data[['Sales', 'Quantity', 'Discount']].values

In [ ]:
def softmax(z):
    exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

In [ ]:
def cross_entropy_loss(y, y_hat):
    m = y.shape[0]
    epsilon = 1e-15  # Минимальное значение для предотвращения log(0)
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)  # Ограничение значений y_hat
    log_likelihood = -np.log(y_hat[np.arange(m), y])
    return np.sum(log_likelihood) / m


def gradient_descent(X, y, learning_rate=0.01, iterations=1000):
    m, n = X.shape
    W = np.random.rand(n, len(np.unique(y)))
    b = np.zeros((1, len(np.unique(y))))

    for i in range(iterations):
        z = np.dot(X, W) + b
        y_hat = softmax(z)

        # One-hot encoding для y
        y_one_hot = np.eye(len(np.unique(y)))[y]

        # Вычисляем ошибку
        error = y_hat - y_one_hot

        # Вычисляем градиенты
        dW = np.dot(X.T, error) / m
        db = np.sum(error, axis=0) / m

        # Обновляем параметры
        W -= learning_rate * dW
        b -= learning_rate * db

        if i % 100 == 0:
            print(f"Итерация {i}, функция потерь: {cross_entropy_loss(y, y_hat)}")

    return W, b


In [ ]:
np.random.seed(42)  # Для воспроизводимости
indices = np.arange(features.shape[0])
np.random.shuffle(indices)

# Разделение индексов для обучающего и тестового наборов
split_idx = int(len(indices) * 0.8)  # 80% данных для обучения, 20% для тестирования

train_indices = indices[:split_idx]
test_indices = indices[split_idx:]




# Создание обучающих и тестовых наборов
X_train, X_test = features[train_indices], features[test_indices]
y_train, y_test = segment_labels[train_indices], segment_labels[test_indices]

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


In [ ]:
W, b = gradient_descent(X_train, y_train, learning_rate=0.01, iterations=1000)

Итерация 0, функция потерь: 1.1923027555721792
Итерация 10, функция потерь: 1.180647496576897
Итерация 20, функция потерь: 1.1697557628894066
Итерация 30, функция потерь: 1.1595821635707448
Итерация 40, функция потерь: 1.1500830722046085
Итерация 50, функция потерь: 1.1412167045596695
Итерация 60, функция потерь: 1.1329431690993461
Итерация 70, функция потерь: 1.1252244934181181
Итерация 80, функция потерь: 1.118024629656654
Итерация 90, функция потерь: 1.1113094418491078
Итерация 100, функция потерь: 1.1050466780028267
Итерация 110, функция потерь: 1.0992059295192853
Итерация 120, функция потерь: 1.0937585803488923
Итерация 130, функция потерь: 1.088677748042833
Итерация 140, функция потерь: 1.0839382186314843
Итерация 150, функция потерь: 1.0795163770284582
Итерация 160, функция потерь: 1.0753901344374446
Итерация 170, функция потерь: 1.071538854029686
Итерация 180, функция потерь: 1.0679432759657495
Итерация 190, функция потерь: 1.0645854426578696
Итерация 200, функция потерь: 1.061

In [ ]:
def predict(X, W, b):
    z = np.dot(X, W) + b
    probabilities = softmax(z)
    return np.argmax(probabilities, axis=1)  # Возвращаем индексы максимальных вероятностей

# Получаем предсказания на тестовом наборе
predictions = predict(X_test, W, b)

# Вычисляем точность как долю правильно предсказанных меток
accuracy = np.mean(predictions == y_test)
print(f"Точность модели: {accuracy:.4f}")

Точность модели: 0.5153
